# Presenting Model Performance


## Importing necessary libraries

In [ ]:
import torch
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from SentimentModel import SentimentModel
import random

## Creating a tokenizer, and methods for tokenizing the data

In [2]:
tokenizer = get_tokenizer('basic_english')


def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)


def tokenize_and_transform(sentence, vocab, tokenizer):
    tokens = tokenizer(sentence)
    token_ids = [vocab[token] for token in tokens]
    return torch.tensor(token_ids, dtype=torch.long).unsqueeze(0)

## Loading the data and building vocab

In [3]:
train_data = list(IMDB(split='train'))
random.shuffle(train_data)

vocab = build_vocab_from_iterator(yield_tokens(data_iter for label, data_iter in train_data), specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

## Setting parameters for model initialization

In [4]:
d_model = 512
num_heads = 8
max_seq_len = 5000
d_ff = 2048
learning_rate = 0.001
num_classes = 2
vocab_size = len(vocab)
num_epochs = 100

## Creating the model and loading trained state from .pth file

In [5]:
model = SentimentModel(d_model, d_ff, num_heads, max_seq_len, num_classes, vocab_size)
model.load_state_dict(torch.load('sentiment_model_state.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [16]:
sentences = []
sentences.append("I saw Hunger Games recently. And I was disappointed by the role played by the main actor. "
            "The movie is badly produced over all")
sentences.append("For the whole duration of the movie I was on the edge of my seat. It was one of the best movies I have ever seen")
sentences.append("My visit to the cinema was an underwhelming experience. I honestly expected more from this director")
sentences.append("I would recommend this series to everyone. Every episode brings something new. You just can't stop watching it!")

In [17]:
for sentence in sentences:
    processed_sentence = tokenize_and_transform(sentence, vocab, tokenizer)
    prediction = model(processed_sentence)
    prediction = torch.sigmoid(prediction)
    sentiment = "positive" if prediction.item()>0.5 else "negative"
    if sentiment == "positive":
        print(f"Sentence: {sentence} was classified as: \033[32m{sentiment}\033[0m\n")
    else:
        print(f"Sentence: {sentence} was classified as: \033[31m{sentiment}\033[0m\n")


Sentence: I saw Hunger Games recently. And I was disappointed by the role played by the main actor. The movie is badly produced over all was classified as: negative

Sentence: For the whole duration of the movie I was on the edge of my seat. It was one of the best movies I have ever seen was classified as: positive

Sentence: My visit to the cinema was an underwhelming experience. I honestly expected more from this director was classified as: negative

Sentence: I would recommend this series to everyone. Every episode brings something new. You just can't stop watching it! was classified as: positive

